This notebook does some preprocessing in the training and test data and save the preprocessed data to pickle files

- The data is loaded from the .csv files and checked for NaN
- Several techniques were tested to replace NaNs, including deleting columns with more than X Nans rows when Columns had less than X Nans.
  Ultimately, the technique that workeed best was to replace NaNs with numeric values according to the feature itself. 
  So, the replacements were as follows: 
       - Zero if the feature was a room area
       - Mode (most common) if the feature was something that every house has, such as utilities
       - Replace with 'NA' or 'None' for categorical features. We assumed that NaN represented houses without some feature,
       such as basement, garage, etc
       - For the Zoning feature, we assumed a correlation with Neighborhood feature and replace NaNs in the test set with the most common
       Zoning in the training set for the same Neighborhood
      
- To encode the categorical features I tested One-Hot Encoding for nominal variables and Ordinal Encoding for ordinal variables. This technique
did not work well since the test set had ordinal categories that did not appear the the training set and it created too many features.
  Ultimately, I decided to go with manual encoding for ordinal features, mapping according to the variables described in the data_description.txt.
  And for categorical features with nominal variables, I applied the LabelEncoder.

- This notebook has the option of removing outliers in the training data using LocalOutlierFactor. The n_neighbors parameter was set to 200 since it
produced the best result. Removing outliers improved the result

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.neighbors import LocalOutlierFactor
from numpy import percentile

pd.set_option('display.max_columns', None)

save_file_train='train_data_2906_ro.pkl'
save_file_test='test_data_2906_ro.pkl'

#remove outliers 'yes' / 'no'

ro='yes'

In [125]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

df=df.set_index('Id')


In [126]:
#convert integers to float
for col in list(df.loc[:,df.dtypes==int].columns):
    df[col] = df[col].astype(float)
    
for col in list(df_test.loc[:,df_test.dtypes==int].columns):
    df_test[col] = df_test[col].astype(float)   

In [127]:
#get column names of columns with Nan
nanfeat=df.columns[df.isna().any()].tolist()
print(nanfeat)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [128]:
#decide what to do with Nans based on features

#for 'GarageYrBlt', 'MasVnrArea', replace with 0
df.GarageYrBlt.fillna(0, inplace=True)
df.MasVnrArea.fillna(0, inplace=True)
df.LotFrontage.fillna(0, inplace=True)

#for categorical features, replace with 'NA' or None
df.Alley.fillna('NA', inplace=True)
df.MasVnrType.fillna('None', inplace=True)
df.BsmtQual.fillna('NA', inplace=True)
df.BsmtCond.fillna('NA', inplace=True)
df.BsmtExposure.fillna('NA', inplace=True)
df.BsmtFinType1.fillna('NA', inplace=True)
df.BsmtFinType2.fillna('NA', inplace=True)
df.FireplaceQu.fillna('NA', inplace=True)
df.GarageType.fillna('NA', inplace=True)
df.GarageFinish.fillna('NA', inplace=True)
df.GarageQual.fillna('NA', inplace=True)
df.GarageCond.fillna('NA', inplace=True)
df.PoolQC.fillna('NA', inplace=True)
df.Fence.fillna('NA', inplace=True)
df.MiscFeature.fillna('NA', inplace=True)

#for general characteristics replace with mode, the most common value
df.Electrical.fillna(df.Electrical.mode()[0], inplace=True) 

In [129]:
#get column names of columns with Nan in test set
test_nanfeat=df_test.columns[df_test.isna().any()].tolist()
print(test_nanfeat)

['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']


In [130]:
#decide what to do with Nans based on features
#for 'GarageYrBlt', 'MasVnrArea', replace with 0
df_test.GarageYrBlt.fillna(0, inplace=True)
df_test.MasVnrArea.fillna(0, inplace=True)
df_test.LotFrontage.fillna(0, inplace=True)
df_test.BsmtFinSF1.fillna(0, inplace=True)
df_test.BsmtFinSF2.fillna(0, inplace=True)
df_test.BsmtUnfSF.fillna(0, inplace=True)
df_test.TotalBsmtSF.fillna(0, inplace=True)
df_test.BsmtFullBath.fillna(0, inplace=True)
df_test.GarageCars.fillna(0, inplace=True)
df_test.GarageArea.fillna(0, inplace=True)
df_test.BsmtHalfBath.fillna(0, inplace=True)

#for categorical features, replace with 'NA' or None
df_test.Alley.fillna('NA', inplace=True)
df_test.MasVnrType.fillna('None', inplace=True)
df_test.BsmtQual.fillna('NA', inplace=True)
df_test.BsmtCond.fillna('NA', inplace=True)
df_test.BsmtExposure.fillna('NA', inplace=True)
df_test.BsmtFinType1.fillna('NA', inplace=True)
df_test.BsmtFinType2.fillna('NA', inplace=True)
df_test.FireplaceQu.fillna('NA', inplace=True)
df_test.GarageType.fillna('NA', inplace=True)
df_test.GarageFinish.fillna('NA', inplace=True)
df_test.GarageQual.fillna('NA', inplace=True)
df_test.GarageCond.fillna('NA', inplace=True)
df_test.PoolQC.fillna('NA', inplace=True)
df_test.Fence.fillna('NA', inplace=True)
df_test.MiscFeature.fillna('NA', inplace=True)


#for general characteristics replace with mode, the most common value
df_test.Utilities.fillna(df_test.Utilities.mode()[0], inplace=True)
df_test.Exterior1st.fillna(df_test.Exterior1st.mode()[0], inplace=True)
df_test.Exterior2nd.fillna(df_test.Exterior2nd.mode()[0], inplace=True)
df_test.KitchenQual.fillna(df_test.KitchenQual.mode()[0], inplace=True)
df_test.Functional.fillna(df_test.Functional.mode()[0], inplace=True)
df_test.SaleType.fillna(df_test.SaleType.mode()[0], inplace=True)

#Zoning should correlate with Neighborhood so use the most common within the neighborhood of the tra
map_zoning=dict(zip(list(df.groupby(['Neighborhood'])['MSZoning'].agg(pd.Series.mode).index),list(df.groupby(['Neighborhood'])['MSZoning'].agg(pd.Series.mode))))
df_test.MSZoning=df_test.Neighborhood
df_test.MSZoning = df_test.MSZoning.map(map_zoning)

In [131]:
#Check for Nan under the dataframes
print(df.isnull().values.any())
print(df_test.isnull().values.any())

False
False


In [133]:
#use manual encoding for rank features
cat1=['Ex','Gd','TA','Fa','Po','NA']
cat2=['GLQ','ALQ','BLQ','Rec','LwQ','Unf','NA']

rank_cols1=[]
rank_cols2=[]
for col in df.loc[:,df.dtypes==object].columns:
    if df[col].str.match( r'^Ex$|^TA$|^Fa$|^Po$').any()==True:
        rank_cols1.append(col)
    elif df[col].eq( r'^GLQ$|^ALQ$|^BLQ$|^Rec$|^LwQ$|^Unf$').any()==True:
        rank_cols2.append(col)

#create mapping
map1=dict(zip(cat1,np.sort(np.arange(len(cat1)))[::-1]))
map2=dict(zip(cat2,np.sort(np.arange(len(cat2)))[::-1]))

#replace cols
for col in rank_cols1:
    df[col] = df[col].map(map1)
    df_test[col] = df_test[col].map(map1)
for col in rank_cols2:
    df[col] = df[col].map(map2)
    df_test[col] = df_test[col].map(map2)

In [134]:
cat3=['Gd','Av','Mn','No','NA']
map3=dict(zip(cat3,np.sort(np.arange(len(cat3)))[::-1]))
#replace cols
df['BsmtExposure'] = df['BsmtExposure'].map(map3)
df_test['BsmtExposure'] = df_test['BsmtExposure'].map(map3)

cat4=['GdPrv','MnPrv','GdWo','MnWw','NA']
rank_cols4=['Fence']
map4=dict(zip(cat4,np.sort(np.arange(len(cat4)))[::-1]))

for col in rank_cols4:
    df[col] = df[col].map(map4)
    df_test[col] = df_test[col].map(map4)

In [135]:
#Use label encoder in non rank features
labelencoder = LabelEncoder()

cols_for_le=list(df.loc[:,df.dtypes==object].columns)

for col in cols_for_le:
    df[col]=labelencoder.fit_transform(df[col])

    df_test[col] = df_test[col].map(lambda s: '<unknown>' if s not in labelencoder.classes_ else s)
    labelencoder.classes_ = np.append(labelencoder.classes_, '<unknown>')

    df_test[col]=labelencoder.transform(df_test[col])

In [136]:
#Check for Nan under the dataframes
print(df.isnull().values.any())
print(df_test.isnull().values.any())

False
False


In [138]:
if ro=='yes':
    #Remove Outliers
    target=df.SalePrice
    df=df.drop('SalePrice', axis=1)
    # identify outliers in the training dataset
    lof = LocalOutlierFactor(n_neighbors=200)
    yhat = lof.fit_predict(df)
    # select all rows that are not outliers
    mask = yhat != -1
    df, target = df[mask], target[mask]

In [139]:
df=pd.concat([df,target],axis=1)
df.to_pickle(save_file_train)
df_test.to_pickle(save_file_test)